In [1]:
import inspect
import importlib.util

module_path = 'ifeature_descriptor.py'
module_name = 'ifeature_descriptor'

spec = importlib.util.spec_from_file_location(module_name, module_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

function_names = [name for name, obj in inspect.getmembers(module) if inspect.isfunction(obj)]
for name in function_names :
    print(name)


AAC
AAINDEX
APAAC
BINARY
BLOSUM62
CKSAAGP
CKSAAP
CTD_C
CTD_D
CTD_T
CTriad
DDE
DPC
EAAC
EGAAC
GAAC
GDPC
GTPC
Geary
Moran
NMBroto
PAAC
QSOrder
SOCNumber
TPC
ZSCALE
checkFasta
minSequenceLength
minSequenceLengthWithNormalAA
readFasta


In [2]:
# ifeature_descriptor.py에서 필요한 함수들을 불러옴
for name in function_names:
    globals()[name] = getattr(module, name)


In [3]:
import sys
import csv
import os

feature_types = [
    
    "AAC",
    "AAINDEX",
    "APAAC",
    "BINARY",
    "BLOSUM62",
    "CKSAAGP",
    "CKSAAP",
    "CTD_C",
    "CTD_D",
    "CTD_T",
    "CTriad",
    "DDE",
    "DPC",
    "EAAC",
    "EGAAC",
    "GAAC",
    "GDPC",
    "GTPC",
    "Geary",
    "Moran",
    "NMBroto",
    "PAAC",
    "QSOrder",
    "SOCNumber",
    "TPC"
]
FEATURE_FUNCTIONS = {feature: globals()[feature] for feature in feature_types if feature in globals()}
# 피처 타입과 해당 함수를 매핑하는 딕셔너리


In [4]:
finished = [
    "TPC"
            
]

selected_items = [i for i, feature in enumerate(feature_types) if feature not in finished]
print(selected_items)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [39]:
note = '''

"" save to db iFeature nomalization ""
---------------------------------------------------------------

amp_file : 분석하고자 하는 파일 이름 (확장자명 제외)
amp_folder : 분석하고자 하는 파일이 들어있는 폴더 이름 (SH 안)

feature_save_folder : 추출한 특징을 저장하고자 하는 폴더 이름 (SH 안)

nor_db_name : 저장하고자 하는 DB 스키마명, DB에서 스키마를 만든 후 실행하면 됨

'''

'''-----------------------------------------------------------'''

amp_file            = 'lg_long'
amp_folder          = 'amp_90'

feature_save_folder = 'lg_long_90_if'

nor_db_name         = 'lg_long_90_if' 

'''-----------------------------------------------------------'''

import os
import csv
import pandas as pd
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler

# 데이터베이스 설정 정보
host = '127.0.0.1'
user = 'root'
password = ''
charset = 'utf8mb4'
    
# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{nor_db_name}?charset={charset}")

succeed = []


def save_to_db(feature_name, featureCSV_dir, clusted):
    csv_file_path = os.path.join(featureCSV_dir, f"{feature_name}{clusted}.csv")
    data = pd.read_csv(csv_file_path)
    if clusted == "_succ" :
        scaler = MinMaxScaler()
        columns_to_scale = data.columns.drop('Seq')  # 'Seq' 컬럼 제외하여 정규화 진행
        data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])
    
    data.to_sql(name=feature_name, con=engine, if_exists='replace', index=False)
    print(f"Normalized data for {feature_name} has been saved to the database.")

def main():    
    
    fasta_file = f'{amp_folder}/{amp_file}.fasta'  # FASTA 파일 경로
    ampCSV_dir = f'{amp_folder}'  # FASTA 파일 경로
    featureCSV_dir = f'{feature_save_folder}'  # 결과 파일을 저장할 디렉토리

    # amp csv 파일 저장
    save_to_db(amp_file, ampCSV_dir, "")
    
    # fasta 파일 읽기
    fastas = readFasta(fasta_file)
    
    for index in selected_items:
        feature_name = feature_types[int(index)]
        
        try:
            encodings = FEATURE_FUNCTIONS[feature_name](fastas)
            output_file = os.path.join(featureCSV_dir, f"{feature_name}_succ.csv")
            
            with open(output_file, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                first_row = True
                for encoding in encodings:
                    if first_row:
                        encoding[0] = 'Seq'  # 첫 번째 행의 첫 번째 컬럼을 'Seq'로 변경
                        first_row = False
                    else:
                        # 시퀀스 문자열의 끝에 콤마가 있는지 확인하고 제거
                        seq = encoding[0]
                        if seq.endswith(','):
                            encoding[0] = seq[:-1]
                    writer.writerow(encoding)
            
            print(f"{feature_name} features have been saved to: {output_file}")
            succeed.append(feature_name)
            save_to_db(feature_name, featureCSV_dir, "_succ")  # 성공한 특징을 데이터베이스에 저장
            

        except Exception as e:
            if os.path.exists(output_file):
                os.remove(output_file)
            
            error_file = os.path.join(featureCSV_dir, f"{feature_name}_error.csv")
            with open(error_file, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Error'])
                writer.writerow([str(e)])
            
            print(f"Error processing {feature_name}. Details saved to: {error_file}")

if __name__ == '__main__':
    main()
    
print("\n\n[ Succed iFeature ]")
for name in succeed:
    print(name)

Normalized data for lg_long has been saved to the database.
AAC features have been saved to: /Users/kimsohui/Project/bioinformatics/Data/SH/lg_long_90_if/AAC_succ.csv
Normalized data for AAC has been saved to the database.
Error: for "AAINDEX" encoding, the input fasta sequences should be with equal length. 


Error processing AAINDEX. Details saved to: /Users/kimsohui/Project/bioinformatics/Data/SH/lg_long_90_if/AAINDEX_error.csv
Error processing APAAC. Details saved to: /Users/kimsohui/Project/bioinformatics/Data/SH/lg_long_90_if/APAAC_error.csv
Error: for "BINARY" encoding, the input fasta sequences should be with equal length. 


Error processing BINARY. Details saved to: /Users/kimsohui/Project/bioinformatics/Data/SH/lg_long_90_if/BINARY_error.csv
Error: for "BLOSUM62" encoding, the input fasta sequences should be with equal length. 


Error processing BLOSUM62. Details saved to: /Users/kimsohui/Project/bioinformatics/Data/SH/lg_long_90_if/BLOSUM62_error.csv
CKSAAGP features have 

In [ ]:
# AAindex

# -*- coding: ms949 -*-
import MyUtil
import copy
import DBPATH

# 아미노산 순서 정의
AA_ORDER1 = [ 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I' ]
AA_ORDER2 = [ 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V'  ]

'''
AAIndex2와 AAIndex3은 다른 파일에서 처리될 예정임을 나타냅니다.
'''

class AAIndex1:

	path = DBPATH.DB_PATH + '/AAindex/aaindex1.txt'
	box = {}

	def __init__(self):
		self.box = {}
		self.__load(self.path)

	# 박스에 저장된 속성의 개수를 반환합니다.
	def getPropertySize(self):
		return len(self.box)

	# 사용 가능한 모든 속성의 이름을 반환합니다.
	def getPropertyNames(self):
		return self.box.keys()

	# 주어진 이름의 속성 값을 반환합니다.
	def getPropertyOf(self, name):
		return self.box[name]

	# 시퀀스에 대한 특정 속성의 점수를 계산합니다.
	def calculatePropertyOfSequence(self, name, sequence):
		sequence = sequence.strip()
		prop = self.getPropertyOf(name)
		score = 0
		for a in sequence:
			score += prop[a]
		return score
	
	# 시퀀스에 대한 특정 속성의 정규화된 점수를 계산합니다.
	def calculateNormalizedPropertyOfSequence(self, name, sequence):
		sequence = sequence.strip()
		score = self.calculatePropertyOfSequence(name, sequence)
		return score / len(sequence)

	# 파일에서 데이터를 로드하는 내부 메서드입니다.
	def __load(self, fname):

		self.box = {}
		f=open(fname,'r')

		name = None
		tmp = {}
		index = -1

		for s in f.readlines():

			s = s.replace('\n', '')

			if len(s)>0:
				if s[:2] == '//':  # 파일의 끝을 나타냅니다.
					if name is not None:
						# 이전에 처리한 속성을 box에 저장합니다.
						self.box[name] = copy.copy(tmp)

						name = None
						tmp = {}
						index=-1

				elif s[0] == 'D':  # 속성 이름의 시작을 나타냅니다.
					name = s[1: s.find('(')].strip()  # 속성 이름을 추출합니다.
					name = name.replace(' ', '_')

				elif index == 1:
					# 두 번째 아미노산 순서에 대한 속성 값을 처리합니다.
					s = s.strip()
					x = s.split(' ')
					i = 0
					for v in x:
						if len(v) != 0:
							if v == 'NA':  # 값이 없는 경우, 처리를 중단합니다.
								name = None
								break
							else:
								# 값이 있는 경우, tmp에 저장합니다.
								tmp[ AA_ORDER2[i] ] = eval(v)
								i += 1
					index = -1

				elif index == 0:
					# 첫 번째 아미노산 순서에 대한 속성 값을 처리합니다.
					s = s.strip()
					x = s.split(' ')
					i = 0
					for v in x:
						if len(v) != 0:
							if v == 'NA':  # 값이 없는 경우, 처리를 중단합니다.
								name = None
								break
							else:
								# 값이 있는 경우, tmp에 저장합니다.
								tmp[ AA_ORDER1[i] ] = eval(v)
								i += 1

					index = 1

				elif s[0] == 'I':
					index = 0

		f.close()

# 메인 실행 블록
if __name__ =='__main__':
	i1 = AAIndex1()
	print i1.getPropertySize()  # 저장된 속성의 크기를 출력합니다.
	for p in i1.getPropertyNames():  # 사용 가능한 모든 속성 이름을 출력합니다.
		print(p)
	
	# 'Positive_charge' 속성에 대한 정보를 출력합니다.
	print(i1.getPropertyOf('Positive_charge'))  
	# 주어진 시퀀스에 대해 'Positive_charge' 속성의 점수를 계산하여 출력합니다.
	print(i1.calculatePropertyOfSequence('Positive_charge', 'AACCCIIIEERRRPPPIILLQQWWEEE'))
	# 주어진 시퀀스에 대해 'Positive_charge' 속성의 정규화된 점수를 계산하여 출력합니다.
	print(i1.calculateNormalizedPropertyOfSequence('Positive_charge', 'AACCCIIIEERRRPPPIILLQQWWEEE'))


In [ ]:
import mysql.connector
import csv
from AAIndex1 import AAIndex1  # AAIndex1 클래스를 가져옵니다.

# MySQL 데이터베이스 연결 설정
db_config = {
    'user': 'your_username',
    'password': 'your_password',
    'host': 'localhost',
    'database': 'your_database_name'
}

# AAIndex1 인스턴스 생성
aaindex = AAIndex1()

# 분석할 특성 이름
property_name = 'Positive_charge'

# CSV 파일로 저장할 데이터 준비
csv_data = [['Sequence', property_name, property_name + '_normalized']]

# MySQL 데이터베이스 연결
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# 펩타이드 시퀀스를 가져올 SQL 쿼리
query = "SELECT sequence_column FROM your_sequence_table"

try:
    cursor.execute(query)
    
    # 모든 시퀀스를 가져와서 각 시퀀스에 대해 특성 값을 계산
    for (sequence,) in cursor:
        property_value = aaindex.calculatePropertyOfSequence(property_name, sequence)
        normalized_property_value = aaindex.calculateNormalizedPropertyOfSequence(property_name, sequence)
        csv_data.append([sequence, property_value, normalized_property_value])

finally:
    cursor.close()
    conn.close()

# CSV 파일 저장
with open('peptide_properties.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print('CSV file saved with peptide properties.')
